In [1]:
!pip install datasets
!pip install fugashi  # for tokenizer
!pip install ipadic   # for tokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=369d198102d096f473243a121444740ea6b488f8623a5c0d0209c1295a0b81f3
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic


In [2]:
import torch
from transformers import AutoTokenizer, BertForSequenceClassification, set_seed, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from datasets import load_from_disk

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import numpy as np
import random
import pprint


In [3]:
### Set seed
# [seed固定+dataloader](https://qiita.com/north_redwing/items/1e153139125d37829d2d)
# parserなどで指定
#################################
seed = 0

#
random.seed(seed)
np.random.seed(seed)

# torch seed
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# huggingface seed
set_seed(seed)

In [4]:
import os
from google.colab import drive
drive.mount('/content/drive')
!ls

Mounted at /content/drive
drive  sample_data


In [5]:
### Hyper parameters
n_labels = 7
# zoo_tag_to_idx = {'CS':0, 'LEGAL':1, 'NEWS':2, 'CODE':3, 'MATH':4, 'Other':5} # 6
zoo_tag_to_idx = {'CS':0, 'LEGAL':1, 'MED':2, 'NEWS':3, 'CODE':4, 'MATH':5, 'Other':6} # 7
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cuda:0"   # specify gpu

epochs = 3#50
batch_size = 16 # Colab batch_size=16->VRAM12.9G

In [6]:
save_datasetDict_dir_path = "/content/drive/MyDrive/Colab Notebooks/MyProjects/test_LLM/test_jsonl/mix_7categories_dataset"
loaded_datasetDict = load_from_disk(save_datasetDict_dir_path)
print(loaded_datasetDict)
train_ds = loaded_datasetDict['train']
#train_ds = train_ds.shuffle(seed=seed).select(range(1000)) # TODO sampling for debug
test_ds = loaded_datasetDict['test']
print(train_ds)
print(test_ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'zoo_tag'],
        num_rows: 6400
    })
    test: Dataset({
        features: ['text', 'zoo_tag'],
        num_rows: 700
    })
})
Dataset({
    features: ['text', 'zoo_tag'],
    num_rows: 6400
})
Dataset({
    features: ['text', 'zoo_tag'],
    num_rows: 700
})


In [7]:
# Prepare Dataset

tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")

def set_label_Idx_from_tag(example, tag_to_idx_dict):
  """ labels にzoo_tagカテゴリを使用し，そのidxの値をset """
  zoo_tag = example['zoo_tag']
  zoo_tag_idx = tag_to_idx_dict.get(zoo_tag, None)
  if zoo_tag_idx is None:
    print(f'Error no zoo_tag {example}, in {tag_to_idx_dict}')
    exit()
  example['labels'] = zoo_tag_idx
  return example

def encode(examples):
  return tokenizer(examples["text"], truncation=True, padding="max_length")


train_ds = train_ds.map(set_label_Idx_from_tag, fn_kwargs={'tag_to_idx_dict': zoo_tag_to_idx})
train_ds = train_ds.map(encode, batched=True)
print(train_ds[0].keys())
train_ds.set_format(type="torch", columns=['labels', 'input_ids', 'token_type_ids', 'attention_mask'])
print(train_ds[0].keys())
print(train_ds[0]['labels'])
print(train_ds[0]['attention_mask'])
print(train_ds[0]['input_ids'])


test_ds = test_ds.map(set_label_Idx_from_tag, fn_kwargs={'tag_to_idx_dict': zoo_tag_to_idx})
test_ds = test_ds.map(encode, batched=True)
test_ds.set_format(type="torch", columns=['labels', 'input_ids', 'token_type_ids', 'attention_mask'])

train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <class '__main__.ColabKernelApp'>.
  StockPickler.save(self, obj, save_persistent_id)
/usr/local/lib/python3.10/dist-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <class '__main__.ColabKernelApp'>: __main__.ColabKernelApp has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)
Parameter 'function'=<function set_label_Idx_from_tag at 0x7e62f2ca0790> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/6400 [00:00<?, ? examples/s]

Map:   0%|          | 0/6400 [00:00<?, ? examples/s]

dict_keys(['text', 'zoo_tag', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'token_type_ids', 'attention_mask'])
tensor(0)
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

In [8]:
# Model
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定 TODO
    num_labels = n_labels, # ラベル数
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
    #attention_probs_dropout_prob=0.3,
    hidden_dropout_prob=0  #ドロップアウト率の設定
)
print('model type: ', type(model))
model.to(device)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model type:  <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [9]:
from tqdm import tqdm   # progress bar in console


total_steps = len(train_dataloader) * epochs
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

def train(model_, dataloader_, optimizer_, scheduler_, device_):
  # Tracking variables.
  predictions_labels = []
  true_labels = []
  total_loss = 0

  model_.train()  # Put the model into training mode.

  for batch in tqdm(dataloader_, total=len(dataloader_)):
      true_labels += batch['labels'].numpy().flatten().tolist()

      # move batch to device
      batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}
      # print('batch: ', batch)   # debug

      model_.zero_grad()  # optimizer_.zero_grad()とほぼ一緒
      outputs = model_(**batch)
      # print('outputs', outputs) # debug

      loss, logits = outputs[:2]
      # print('loss: ', loss)   # debug

      total_loss += loss.item()
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model_.parameters(), 1.0)

      optimizer_.step()
      scheduler_.step()

      logits = logits.detach().cpu().numpy()

      # Convert these logits to list of predicted labels values.
      predictions_labels += logits.argmax(axis=-1).flatten().tolist()

  # Calculate the average loss over the training data.
  avg_epoch_loss = total_loss / len(dataloader_)

  # Return all true labels and prediction for future evaluations.
  return true_labels, predictions_labels, avg_epoch_loss

def validation(model_, dataloader_, device_):
  predictions_labels = []
  true_labels = []
  total_loss = 0

  model_.eval()

  for batch in tqdm(dataloader_, total=len(dataloader_)):
      true_labels += batch['labels'].numpy().flatten().tolist()

      # move batch to device
      batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}
      with torch.no_grad():
          outputs = model_(**batch)
          loss, logits = outputs[:2]
          logits = logits.detach().cpu().numpy()
          total_loss += loss.item()

          predict_content = logits.argmax(axis=-1).flatten().tolist()
          predictions_labels += predict_content

  avg_epoch_loss = total_loss / len(dataloader_)

  # Return all true labels and prediciton for future evaluations.
  return true_labels, predictions_labels, avg_epoch_loss

### Zero-shot　TEST
valid_labels, valid_predict, valid_loss = validation(model, test_dataloader, device)
val_acc = accuracy_score(valid_labels, valid_predict)
print(f'[Zero-shot TEST] loss: {valid_loss:.3f}, acc: {val_acc:.3f}')


### FINE TUNING: Loop through each epoch.
print('Start Training...')
for epoch in tqdm(range(1, epochs+1)):
  print(f' epoch {epoch} / {epochs}')
  train_labels, train_predict, train_loss = train(model, train_dataloader, optimizer, scheduler, device)
  train_acc = accuracy_score(train_labels, train_predict)
  print(f'[train] loss: {train_loss:.3f}, acc: {train_acc:.3f}')

  valid_labels, valid_predict, valid_loss = validation(model, test_dataloader, device)
  val_acc = accuracy_score(valid_labels, valid_predict)
  print(f'[test] loss: {valid_loss:.3f}, acc: {val_acc:.3f}')

print('training end.')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 44/44 [00:22<00:00,  2.00it/s]


[Zero-shot TEST] loss: 2.021, acc: 0.144
Start Training...


  0%|          | 0/3 [00:00<?, ?it/s]

 epoch 1 / 3



100%|██████████| 400/400 [10:07<00:00,  1.52s/it]


...

[test] loss: 0.010, acc: 0.999
 epoch 3 / 3



100%|██████████| 400/400 [10:09<00:00,  1.52s/it]


[train] loss: 0.001, acc: 1.000



100%|██████████| 3/3 [31:39<00:00, 633.09s/it]

[test] loss: 0.011, acc: 0.999
training end.
